# Model Evaluation & Registry
## Evaluate model metrics & Manage artiacts for trained models  

In [38]:
import os
import sagemaker
import logging
import boto3
import time
import pandas as pd
import json
import botocore
from botocore.exceptions import ClientError


# ========================== low-level service client of the boto3 session ==========================
config = botocore.config.Config(user_agent_extra='bedissj-1699438736259')


sm = boto3.client(service_name='sagemaker', 
                  config=config)

sm_runtime = boto3.client('sagemaker-runtime',
                          config=config)

sess = sagemaker.Session(sagemaker_client=sm,
                         sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name


## 1. Evaluate Model Metrics for Built-in AWS SageMaker Algorithms

In [48]:
# ========================== Processing parameters ==========================
FRAMEWORK_VERSION = '1.2-1'
instance_type = 'ml.t3.medium'
instance_count = 1

model_s3_uri = 's3://{}/sagemaker-scikit-lea-240604-1642-002-59cbda6b/output/'.format(bucket)
test_data_s3_uri = 's3://{}/sagemaker-scikit-learn-2024-03-06-21-05-51-569/output/bank-churn-test/'.format(bucket)


In [49]:
from sagemaker.processing import ProcessingInput, ProcessingOutput


input_model = '/opt/ml/processing/input/model'
input_data = '/opt/ml/processing/input/data'
output_data = '/opt/ml/processing/output'


# ========================== Processing inputs ==========================
processing_inputs = [
    #### Trained model ####
    ProcessingInput(input_name= 'model',
                    source=model_s3_uri,
                    destination=input_model),
    
    #### Test subset ####
    ProcessingInput(input_name='testSubset',
                    source=test_data_s3_uri,
                    destination=input_data)
]


# ========================== Processing outputs ==========================
processing_outputs = [
    ProcessingOutput(output_name='metrics',
                     source=output_metrics,
                     s3_upload_mode='EndOfJob')

]


# ========================== Processing arguments ==========================
processing_args=[
    '--input-model', str(input_model),
    '--input-data', str(input_data),
    '--output-data', str(output_data)
]


In [62]:
from sagemaker.sklearn.processing import SKLearnProcessor


evaluation_processor = SKLearnProcessor(
    framework_version=FRAMEWORK_VERSION,
    instance_type=instance_type,
    instance_count=instance_count,
    env={'AWS_DEFAULT_REGION': region},
    role=role,
)


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [63]:
evaluation_processor.run(
    code='src/evaluate_metrics.py',
    inputs=processing_inputs,
    outputs=processing_outputs,
    arguments=processing_args,
    wait=False
)


INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2024-06-13-14-51-43-046


In [64]:
evaluation_processor.latest_job.wait(logs=True)


....................................................................................................Collecting matplotlib==3.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 20.0 MB/s eta 0:00:00


UnexpectedStatusException: Error for Processing job sagemaker-scikit-learn-2024-06-13-14-51-43-046: Failed. Reason: AlgorithmError: See job logs for more information

In [44]:
from pprint import pprint


pprint(evaluation_processor.latest_job.describe(), compact=True)

{'AppSpecification': {'ContainerArguments': ['--input-model',
                                             '/opt/ml/processing/input/model',
                                             '--input-data',
                                             '/opt/ml/processing/input/data',
                                             '--output-data',
                                             '/opt/ml/processing/output'],
                      'ContainerEntrypoint': ['python3',
                                              '/opt/ml/processing/input/code/evaluate_metrics.py'],
                      'ImageUri': '659782779980.dkr.ecr.eu-west-3.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3'},
 'CreationTime': datetime.datetime(2024, 6, 13, 11, 6, 7, 165000, tzinfo=tzlocal()),
 'Environment': {'AWS_DEFAULT_REGION': 'eu-west-3'},
 'FailureReason': 'AlgorithmError: See job logs for more information',
 'LastModifiedTime': datetime.datetime(2024, 6, 13, 11, 22, 44, 740000, tzinfo=tzlocal()),
 'Proc

## 2. Evaluate Model Metrics for models trained with custom script